In [3]:
# !pip install selenium

In [39]:
import time
import requests
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd

## 페이지 가져오기

In [6]:
channel_link = 'https://www.youtube.com/@RawFishEater/videos'
driver = webdriver.Chrome()
driver.get(channel_link)

# 영상 링크 배열에 저장하기
video_links = set()
scroll_count = 0

for _ in range(19):  # 20으로 설정해야함
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(1)
    scroll_count += 1

    videos = driver.find_elements(By.CSS_SELECTOR, '#thumbnail')
    for video in videos:
        href = video.get_attribute('href')
        if href:
            video_links.add(href)

print(scroll_count)
print(len(video_links))

for i, link in enumerate(video_links):
    if i < 10:
        print(link)
    else:
        break

19
442
https://www.youtube.com/watch?v=Zt52lXF3O8A
https://www.youtube.com/watch?v=UiwmFtfIHLU
https://www.youtube.com/watch?v=JVBi4FVrboQ
https://www.youtube.com/watch?v=nI1o12gVPgc
https://www.youtube.com/watch?v=68ejByrDlzg
https://www.youtube.com/watch?v=pwRLFHa7n68
https://www.youtube.com/watch?v=wyphXsNerg8
https://www.youtube.com/watch?v=e-Cj6mdOse4
https://www.youtube.com/watch?v=pFo49JRu-xE
https://www.youtube.com/watch?v=AOrEb9MNASI


In [9]:
# # 배열에 저장한 영상 링크에 접속하기
# for link in video_links:
#     print(link)

In [40]:
tmp_driver = webdriver.Chrome()
tmp_driver.get('https://www.youtube.com/watch?v=SUhtMdjjsAc&t=782s')
tmp_driver.implicitly_wait(3)

expand_button = tmp_driver.find_element(By.ID, "expand")
expand_button.click()
    
description = tmp_driver.find_element(By.CSS_SELECTOR, "#description-inline-expander")
text = description.text
print(text)
print('*' * 30)

restaurant_info = re.search(r'\*식당정보(.*?)(?=\n\*|\Z)', text, re.DOTALL)

# 추출된 정보 출력
if restaurant_info:
    print(restaurant_info.group(1))
else:
    print("식당 정보를 찾을 수 없습니다.")

줄거리 : 서비스로 손님을 사로 잡는 숙성회 전문점

*식당정보
- 식당명 : 무스비
- 식당위치 : 경기 안양시 동안구 인덕원로 12 1층 103호 무스비
- 전화번호 : 0507-1427-0726
- 영업시간 : 18:00-01:00 일요일 휴무, 10/31 휴무
- 리뷰 메뉴 : 숙성회 2인(38,000원), 라구나스텐(20,000원), 탄탄 니꾸우동(17,000원)

#회사랑 #인덕원맛집 #무스비 #숙성회 #웨이팅
음악
노래 3곡
Drag Race
TrackTribe
Drag Race
Jazz Organ Trio Cool Blue
Doug Maxwell/Media Right Productions
Jazz Organ Trio Cool Blue
Bluesed and Abused
Quincas Moreira
Bluesed and Abused
음악
스크립트
스크립트를 보면서 시청하세요.
스크립트 표시
회사랑RawFishEater
구독자 22.8만명
동영상
정보
간략히
******************************

- 식당명 : 무스비
- 식당위치 : 경기 안양시 동안구 인덕원로 12 1층 103호 무스비
- 전화번호 : 0507-1427-0726
- 영업시간 : 18:00-01:00 일요일 휴무, 10/31 휴무
- 리뷰 메뉴 : 숙성회 2인(38,000원), 라구나스텐(20,000원), 탄탄 니꾸우동(17,000원)

#회사랑 #인덕원맛집 #무스비 #숙성회 #웨이팅
음악
노래 3곡
Drag Race
TrackTribe
Drag Race
Jazz Organ Trio Cool Blue
Doug Maxwell/Media Right Productions
Jazz Organ Trio Cool Blue
Bluesed and Abused
Quincas Moreira
Bluesed and Abused
음악
스크립트
스크립트를 보면서 시청하세요.
스크립트 표시
회사랑RawFishEater
구독자 22.8만명
동영상
정보
간략히


In [ ]:


# 영상 링크에서 식당 정보 추출해서 DataFrame에 저장하기

# 저장한 DataFrame을 정렬하고 csv 파일로 만들기